In [44]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import pyautogui
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [45]:
global driver

def init_driver():
    global driver
    profile = webdriver.FirefoxProfile()
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0'

    profile.set_preference("general.useragent.override", user_agent)
    driver = webdriver.Firefox(profile)
    driver.maximize_window()
    time.sleep(2)
    
    return driver

#driver.get("https://www.milenio.com/opinion/valeria-moy/page/1")


In [ ]:
def milenio_url_scrapper(url_1):
    driver = init_driver()
    links = []
    
    driver.get(url_1)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    try:
        driver.find_element_by_xpath('/html/body/div[4]/div/div/button').click()
    except:
        print('No cookies overlay detected.')
    
    while True:
        print(driver.current_url)
        respuesta = driver.page_source
        sopa = BeautifulSoup(respuesta, 'html')        
        titles = sopa.select('[class="title"]', href = True)
        
        for title in titles:            
            links.append('https://www.milenio.com' + title.select('a[href]')[0]['href'])
        try:
            if driver.current_url.split('/')[-1] == '1':
                driver.find_element_by_xpath('/html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a').click()
            else:
                driver.find_element_by_xpath('/html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a[2]').click()
            time.sleep(0.5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        except Exception as e:
            print(e)
            break
        
    driver.quit()
    return links
    

def milenio_url_scrapper(url, pages):
    """This function takes a columnist's page url from the 'Milenio' website 
    and a number of pages and bring backs all the columnists articles links
    Parameters:
    -----------------------------
    
    url: URL to the columnists site, must end before the page number 
    example : 'https://www.milenio.com/opinion/valeria-moy/page/'
    
    pages: Number of pages to be scrapped, the number must exist in the 'Milenio' website"""
    
    links = []
    
    for page in range(1, pages + 1):
        print(url + str(page))
        respuesta = requests.get(url + str(page) + '?').content
        print(respuesta)
        sopa = BeautifulSoup(respuesta, 'html')        
        titles = sopa.select('[class="title"]', href = True)
        
        for title in titles:            
            links.append('https://www.milenio.com' + title.select('a[href]')[0]['href'])
    
    return links

In [82]:
def milenio_content_scrap(link):
    """"""
    respuesta = requests.get(link).content        
    sopa = BeautifulSoup(respuesta, 'html')      
    title = sopa.select('title')[0].text
    try:
        date = sopa.select('div time[class="date"]')[0].text
        body = sopa.select('div [class="media-container opinion"]', id = 'content-body')[0].text
    except Exception as e:
        date = sopa.select('body > div.body-content > div > article > div.content-columns > div.column-right > div.nd-content-body > div.content-date > time')[0].text
        body = sopa.select('#content-body > p:nth-child(1)')[0].text
        print(f'Error on link: {link}')
    
    return title, date, body

In [83]:
def scrap_columnista (url):
    titles = []
    dates = []
    bodies = []
    contador = 1
    
    links = milenio_url_scrapper(url)
    for link in links:
        title, date, body = milenio_content_scrap(link)
        print(f'Procesando: {contador} de {len(links)}')
        titles.append(title)
        dates.append(date)
        bodies.append(body)
        contador += 1
    return {'Title':titles, 'Date':dates, 'Body':bodies}

In [84]:
val_moy = scrap_columnista('https://www.milenio.com/opinion/valeria-moy/page/1')

https://www.milenio.com/opinion/valeria-moy/page/1
1
https://www.milenio.com/opinion/valeria-moy/page/2?
2?
https://www.milenio.com/opinion/valeria-moy/page/3?
3?
https://www.milenio.com/opinion/valeria-moy/page/4?
4?
https://www.milenio.com/opinion/valeria-moy/page/5?
5?
https://www.milenio.com/opinion/valeria-moy/page/6?
6?
https://www.milenio.com/opinion/valeria-moy/page/7?
7?
https://www.milenio.com/opinion/valeria-moy/page/8?
8?
https://www.milenio.com/opinion/valeria-moy/page/9?
9?
https://www.milenio.com/opinion/valeria-moy/page/10?
10?
https://www.milenio.com/opinion/valeria-moy/page/11?
11?
Message: Unable to locate element: /html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a[2]

Procesando: 1 de 99
Procesando: 2 de 99
Procesando: 3 de 99
Procesando: 4 de 99
Procesando: 5 de 99
Procesando: 6 de 99
Procesando: 7 de 99
Procesando: 8 de 99
Procesando: 9 de 99
Procesando: 10 de 99
Procesando: 11 de 99
Procesando: 12 de 99
Procesando: 13 de 99
Procesando: 14 de 99
Procesando: 15 d

In [85]:
pd.DataFrame(val_moy)

,Title,Date,Body
0,Presupuesto,08.09.2020/03:33,\nEn unas horas se presentará el paquete econó...
1,El otro informe,01.09.2020/02:50,\nEstamos acostumbrados a escuchar todos los p...
2,Problemas colectivos y comportamiento individual,25.08.2020/00:43,\nLlama la atención el debate que se ha desenc...
3,Burbujas escolares como salvavidas,18.08.2020/02:56,\nNo cabe duda de que muchos de los temas econ...
4,La flexibilidad de lo informal,11.08.2020/01:56,\nLa informalidad laboral en México es muy for...
...,...,...,...
94,Pero qué necesidad,20.11.2018/01:48,"\nLa idea no era parafrasear a Juan Gabriel, p..."
95,La forma y el fondo,13.11.2018/02:19,\nEl jueves pasado el senador Ricardo Monreal ...
96,Abróchense los cinturones,06.11.2018/01:20,"\nNo, no es solo el aeropuerto. No es solo la ..."
97,El México que he visto,20.02.2018 07:12:38,Creo que es posible vernos de otra forma. Pero...


# El Universal

In [243]:
links = []
ef = "https://www.eluniversal.com.mx/valeria-moy/peras-y-manzanas"
respuesta = requests.get(ef).content        
sopa = BeautifulSoup(respuesta, 'html')        
titles = sopa.select('h3[class="ce3-AutorColumna_Titulo uniplus"]', href = True)
for title in titles:
    links.append("https://www.eluniversal.com.mx"+title.select('a[href]')[0]['href'])
        
titles = sopa.select('h3[class="ce3-AutorColumna_Titulo"]', href = True)
for title in titles:
    links.append("https://www.eluniversal.com.mx"+title.select('a[href]')[0]['href'])
    
titles = sopa.select('[class="ce9-Tipo4_Titulo"]', href = True)
for title in titles:
    links.append("https://www.eluniversal.com.mx"+title.select('a[href]')[0]['href'])
    